In [3]:
from sklearn.model_selection import train_test_split
from transformers import AutoModel, AutoTokenizer 
import pandas as pd
import numpy as np
from tqdm import tqdm
import torch

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
df = pd.read_csv("/kaggle/input/last-dataset/last_dataset.csv")

In [5]:
df.head()

,Unnamed: 0,review,sentiment
0,0,bayiler satar artık e burası türkiye,positive
1,1,fiyatlara bin ekleyin bayide bulamayacaksınız ...,positive
2,2,motor kasaya yakışmış,positive
3,3,tl eksik aracı almam neyse sene bekleyem alırım,positive
4,4,eskiden milyoner olmak vardı derlerdi nerden g...,positive


In [6]:
X_train, X_test = train_test_split(df, test_size=0.25,random_state=42)

In [7]:
X_test

,Unnamed: 0,review,sentiment
36262,36262,yardımlarınız gerçekten teşekkür ederim halen ...,positive
36783,36783,kardeş ben yıldır dizel logan kullanıyorum zat...,positive
32633,32633,ben bir göz gezdirdim bin liraya çıkan modelle...,positive
14564,14564,tdi fiyat alan oldu arkadaşlar bin liste fiyat...,positive
50638,50638,hayırlı olsun,positive
...,...,...,...
14471,14471,model tdi coupe almayı düşünüyorum civarı km g...,negative
48558,48558,avrupa an satılıyor,positive
7831,7831,tsi polonun performansı gayet tatminkar kadar ...,positive
47091,47091,maalesef çalışmadı dizel bir modeline cc olmuy...,negative


In [8]:
mask = X_train['review'].str.len() < 300 
X_train = X_train.loc[mask]

In [9]:
mask = X_test['review'].str.len() < 300 
X_test = X_test.loc[mask]

In [10]:
X_train.shape

(48333, 3)

In [11]:
type(X_train)

pandas.core.frame.DataFrame

In [12]:
X_train["review"]

15962    quoteorijinalden alıntı ankara_ahmet teşekkür ...
6748     quoteorijinalden alıntı fc nin motorunun üstün...
25124    maaşallah arabanız güzelmiş bende aynı özellik...
6420     hayırlı olsun şimdiden hangi bayi lanet gelsin...
9724     uzaktan kumandalı araba galiba motor sesine eb...
                               ...                        
62570    kazanıp ye araççlardan b segment alırsın sadec...
38158    bunlara göre herşey hikaye pandemi çip savaş h...
860      scirocco nun kapısı gelmez mi arkadaşlar güzel...
15795    almanlar bizi kıskandığı fiyatları yüksek tutu...
56422                                             mercedes
Name: review, Length: 48333, dtype: object

In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_name = "dbmdz/bert-base-turkish-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
bert_model = AutoModel.from_pretrained(model_name).to(device)

In [14]:
def feature_extraction(df):
    tokenized = df['review'].apply((lambda x: tokenizer.encode(str(x), add_special_tokens=True)))
    max_len = 0
    for i in tokenized.values:
        if (len(i) > max_len):
            max_len = len(i)
            
    last_hidden_states = []
    padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])
    attention_mask = np.where(padded != 0, 1, 0)
    all_hidden_states = []
    batch_size = 8
    for i in range(0, len(padded), batch_size):
        batch_input_ids = torch.tensor(padded[i:i+batch_size]).to(device)
        batch_attention_mask = torch.tensor(attention_mask[i:i+batch_size]).to(device)

        with torch.no_grad():
#             last_hidden_states += list(bert_model(batch_input_ids, attention_mask=batch_attention_mask)[0][0][0].cpu().numpy())
#             [:,0,:]
            last_hidden_states += list(bert_model(batch_input_ids, attention_mask=batch_attention_mask)[0][:,0,:].cpu().numpy())
#             list(x[0][0].cpu().numpy())
        
#         all_hidden_states.extend(last_hidden_states)
        
#     last_hidden_states[0].size()
#     torch.Size([668, 75, 768])
#     return last_hidden_states[0][:,0,:].numpy()
    return last_hidden_states
    

In [15]:
X_train_tokens = feature_extraction(X_train)

In [16]:
X_test_tokens = feature_extraction(X_test)

In [17]:
len(X_test_tokens)

16112

In [18]:
X_train_tokens[0][0]

1.0083114

In [19]:
len(X_train_tokens[0])

768

In [20]:
len(X_train_tokens[0])

768

In [21]:
matris = np.array(X_train_tokens)

In [22]:
matris.shape

(48333, 768)

In [23]:
type(X_train_tokens)

list

In [24]:
X_train_tokens.shape

AttributeError: 'list' object has no attribute 'shape'

In [101]:
4641792/146

31793.095890410958

In [80]:
X_train_tokens[0][:,0,:].numpy()

TypeError: string indices must be integers

In [91]:
X_train_tokens[0].size()

AttributeError: 'str' object has no attribute 'size'

In [42]:
X_train.shape

(48347, 4)

In [53]:
type(X_train_tokens)

transformers.modeling_outputs.BaseModelOutputWithPoolingAndCrossAttentions

In [62]:
X_train_tokens_np[0].size()

IndexError: too many indices for array: array is 0-dimensional, but 1 were indexed

In [64]:
X_train_tokens_np.shape

()

In [40]:

torch.Size([48347, 75, 768])

torch.Size([48347, 75, 768])

In [39]:
X_train_tokens

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[ 0.0808,  0.0813,  0.2418,  ...,  0.6220, -0.7129,  0.9066],
         [ 0.1090,  0.0168, -0.9655,  ...,  0.6115,  0.4178,  1.4624],
         [-0.4136,  0.1177,  0.3412,  ...,  0.5167, -1.2618,  1.0250],
         ...,
         [ 0.3841,  0.3316, -0.6968,  ...,  0.6313, -0.3202,  0.7639],
         [ 0.5687,  0.1228, -0.6248,  ...,  0.7955, -0.5041,  0.5846],
         [-0.7858, -1.3663, -0.0470,  ...,  0.8300, -0.6014,  0.7653]],

        [[ 0.6223,  0.3800,  0.1528,  ...,  0.1840,  1.0982,  0.9544],
         [-1.6429, -0.4704,  0.9964,  ...,  0.1110, -0.3723,  0.4553],
         [-1.0311, -0.1955,  0.5316,  ...,  0.2848, -0.7035,  0.4927],
         ...,
         [-0.8550, -0.0610, -0.2485,  ...,  0.5861,  0.3587,  0.7029],
         [-0.0691, -0.4977, -0.2881,  ...,  0.4570, -0.5029,  1.5062],
         [ 0.0205, -0.7672, -0.3209,  ...,  0.6431, -0.8487,  1.6601]],

        [[-0.3529,  0.1620,  0.4437,  ...,  0.3400, -

In [22]:
def feature_extraction(token,token_attention_mask):
    with torch.no_grad():
        last_hidden_states = bert_model(token, attention_mask=token_attention_mask)
    last_hidden_states[0].size()
    torch.Size([668, 75, 768])
    return last_hidden_states[:,0,:][0].cpu().numpy()

In [17]:
def tokenization(df,max_len=0):
    tokenized = df['review'].apply((lambda x: tokenizer.encode(str(x), add_special_tokens=True)))
    if max_len == 0:
        for i in tokenized.values:
            if (len(i) > max_len):
                max_len = len(i)
    padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])
    attention_mask = np.where(padded != 0, 1, 0)
    input_ids =  torch.tensor(padded).to(device)
    attention_mask = torch.tensor(attention_mask).to(device)
    return input_ids, attention_mask, max_len

In [45]:
tokenized = df['review'].apply((lambda x: tokenizer.encode(str(x), add_special_tokens=True)))

Token indices sequence length is longer than the specified maximum sequence length for this model (530 > 512). Running this sequence through the model will result in indexing errors


In [50]:
max_len = 0
if max_len == 0:
    for i in tokenized.values:
        if (len(i) > max_len):
            max_len = len(i)
padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])
attention_mask = np.where(padded != 0, 1, 0)

In [52]:
padded.shape

(93991, 3217)

In [86]:
attention_mask = np.where(padded != 0, 1, 0)

In [56]:
input_ids = torch.tensor(padded).to(device)  
attention_mask = torch.tensor(attention_mask).to(device)

In [87]:
input_ids.shape

torch.Size([93991, 3217])

In [94]:
input_ids[1]

tensor([    2, 21817,  2551,  ...,     0,     0,     0], device='cuda:0')

In [59]:
attention_mask.shape

torch.Size([93991, 3217])

In [55]:
attention_mask.shape

(93991, 3217)

In [47]:
tokenized

0        [2, 24030, 1990, 2599, 1978, 3138, 73, 10471, ...
1        [2, 21817, 2551, 14565, 24030, 1988, 7580, 292...
2                    [2, 4866, 13492, 2029, 7931, 2149, 3]
3        [2, 10077, 4679, 4563, 22246, 10469, 4783, 125...
4        [2, 12020, 3015, 1972, 2688, 3869, 14539, 2016...
                               ...                        
93986    [2, 4563, 31400, 4345, 1305, 1009, 5707, 1988,...
93987    [2, 87, 17632, 2816, 4563, 1027, 3424, 7471, 3...
93988                      [2, 2339, 8151, 13344, 1980, 3]
93989    [2, 9714, 3553, 2933, 2090, 20815, 3582, 21239...
93990    [2, 28484, 2956, 3684, 2112, 5484, 4200, 6100,...
Name: review, Length: 93991, dtype: object

In [ ]:
x, _ = bert(torch.stack([torch.tensor(x)]).to(device))
        return list(x[0][0].cpu().numpy())

In [107]:
with torch.no_grad():
    input_ids_tensor = torch.stack([input_ids[1]])
    attention_mask_tensor = torch.tensor([attention_mask[1]])  # attention_mask'ı Tensor olarak dönüştürüyoruz
    last_hidden_states = bert_model(input_ids_tensor, attention_mask=attention_mask_tensor)
    last_hidden_states = last_hidden_states[:, 0, :][0].cpu().numpy()


/tmp/ipykernel_28/2666584752.py:3: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /usr/local/src/pytorch/torch/csrc/utils/tensor_new.cpp:245.)
  attention_mask_tensor = torch.tensor([attention_mask[1]])  # attention_mask'ı Tensor olarak dönüştürüyoruz


RuntimeError: The expanded size of the tensor (3217) must match the existing size (512) at non-singleton dimension 1.  Target sizes: [1, 3217].  Tensor sizes: [1, 512]

In [18]:
X_train_tokens,X_train_attention_mask,max_len = tokenization(X_train)

In [71]:
X_train_tokens.shape

torch.Size([48347, 146])

In [73]:
X_train_attention_mask.shape

torch.Size([48347, 146])

In [19]:
X_test_tokens,X_test_attention_mask,_ = tokenization(X_test,max_len)

In [75]:
X_test_tokens.shape

torch.Size([16098, 146])

In [23]:
y_train

NameError: name 'y_train' is not defined

In [25]:
def process_data(df,tokens):
    mapping = {'negative':0, 'neutral':1, 'positive':2}
    X = []
    y = []
    for idx, review in enumerate(tqdm(df["review"].values)):
        X.append(tokens[idx])
        y_val = np.zeros(3)
        y_val[mapping[df.iloc[idx]['sentiment']]] = 1
        y.append(y_val)
    return np.array(X), np.array(y)

In [26]:
X_train, y_train = process_data(X_train,X_train_tokens)

100%|██████████| 48333/48333 [00:02<00:00, 18770.84it/s]


In [27]:
len(y_train)

48333

In [28]:
X_test, y_test = process_data(X_test, X_test_tokens)

100%|██████████| 16112/16112 [00:00<00:00, 19535.44it/s]


In [29]:
X_test

array([[-0.6205371 ,  1.0695187 ,  0.44955975, ..., -0.23404612,
        -0.42090574,  0.2612578 ],
       [ 0.65037453, -0.1304944 ,  0.18748899, ...,  0.5637733 ,
         1.6496999 , -0.67957747],
       [ 0.44589907,  0.69956934,  0.07245254, ...,  0.26929614,
         1.1619103 ,  1.0208415 ],
       ...,
       [ 0.7238513 , -0.09289359,  0.04365493, ...,  0.40675685,
         1.1015791 , -0.6361791 ],
       [ 0.8309579 , -0.3744982 ,  0.04445816, ...,  0.46860906,
         1.8479006 , -0.62462586],
       [-0.3989065 ,  1.5098075 ,  0.57541347, ..., -0.98508924,
        -0.61116904,  0.7988937 ]], dtype=float32)

In [4]:
import numpy as np

In [9]:
class_counts = [0, 0, 0]
for el in y_train:
    print(el)
    class_counts[np.argmax(el)]+=1

[0, 0, 1]

[0, 1, 0]

[0, 1, 0]

[0, 0, 1]

[1, 0, 0]


In [13]:
np.argmax([1,0,0])

0

In [18]:
class_counts = [0, 0, 0]

In [39]:
class_counts[np.argmax([0,0,1])]+=1

In [40]:
class_counts

[3, 1, 1]

In [41]:
sum(class_counts)

5

In [8]:
y_train = [[0,0,1],
           [0,1,0],
           [0,1,0],
           [0,0,1],
           [1,0,0],
]

In [48]:
class_weights = {idx:sum(class_counts)/el for idx, el in enumerate(class_counts)}

In [49]:
for idx, el in enumerate(class_counts):
    print(idx)
    print(el)

0

3

1

1

2

1


In [47]:
class_weights

{0: 5.0, 1: 5.0, 2: 5.0}

In [37]:
class_weights

{0: 4.0, 1: 4.0, 2: 4.0}

In [34]:
bert_model

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(32000, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

In [1]:
bert_model

NameError: name 'bert_model' is not defined

In [ ]:
embedding_dim = bert.config.to_dict()['hidden_size']

In [1]:
model = Sequential()

NameError: name 'Sequential' is not defined

In [ ]:
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-xl")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-xl")

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration